This is based on 05a not 04_callbacks this is better to understand first.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import torch
import matplotlib.pyplot as plt
from time import sleep

In [12]:
def slow_calculation():
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
    return res

In [11]:
slow_calculation()

30

In [16]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb : cb(i)
    return res

In [48]:
def show_progress(ex,epoch):
    print(f"{ex}! We've finished {epoch}!")

In [18]:
slow_calculation(show_progress)

Awesome! We've finished 0!
Awesome! We've finished 1!
Awesome! We've finished 2!
Awesome! We've finished 3!
Awesome! We've finished 4!


30

In [38]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        res += i*i
        sleep(1)
        if cb : cb(i)
    return res

In [30]:
slow_calculation(lambda o : print(f"Awesome we've finished {o}!"))

Awesome we've finished 0!
Awesome we've finished 1!
Awesome we've finished 2!
Awesome we've finished 3!
Awesome we've finished 4!


30

In [35]:
def make_show_progress(exclamation):
    _inner = lambda epoch : print(f"{exclamation} we'have finished {epoch}")
    return _inner

In [36]:
slow_calculation(make_show_progress('hello'))

hello we'have finished 0
hello we'have finished 1
hello we'have finished 2
hello we'have finished 3
hello we'have finished 4


30

In [39]:
def make_show_progress(exclamation):
    def _inner(epoch) : print(f"{exclamation} we'have finished {epoch}")
    return _inner

In [41]:
slow_calculation(make_show_progress('Hi'))

Hi we'have finished 0
Hi we'have finished 1
Hi we'have finished 2
Hi we'have finished 3
Hi we'have finished 4


30

In [42]:
from functools import partial

In [49]:
slow_calculation(partial(show_progress,('OK')))

OK! We've finished 0!
OK! We've finished 1!
OK! We've finished 2!
OK! We've finished 3!
OK! We've finished 4!


30

In [51]:
class ProgressShowingCallback():
    def __init__(self, exclamation = "meh"): self.exclamation=exclamation
    def __call__(self,epoch):print(f"{self.exclamation} we've finised epoch{epoch}")

In [52]:
cb= ProgressShowingCallback("Psst")

In [53]:
slow_calculation(cb)

Psst we've finised epoch0
Psst we've finised epoch1
Psst we've finised epoch2
Psst we've finised epoch3
Psst we've finised epoch4


30

In [54]:
def slow_calculation(cb=None):
    res = 0
    for i in range(5):
        if cb : cb.before_calc(i)
        res += i*i
        sleep(1)
        if cb : cb.after_calc(i,val=res)
    return res

In [69]:
class PrintStepCallback():
    def __init__(self): pass
    def before_calc(self, *args, **kwargs): print(f"About to start")
    def after_calc (self, *args, **kwargs): print(f"Done step")

In [76]:
class SlowCalculator():
    def __init__(self, cb=None): self.cb,self.res = cb,0
    
    def callback(self, cb_name, *args):
        print
        if not self.cb: return
        cb = getattr(self.cb,cb_name, None)
        if cb: return cb(self, *args)

    def calc(self):
        for i in range(5):
            self.callback('before_calc', i)
            self.res += i*i
            sleep(1)
            if self.callback('after_calc', i):
                print("stopping early")
                break

In [77]:
class ModifyingCallback():
    def after_calc (self, calc, epoch):
        print(f"After {epoch}: {calc.res}")
        if calc.res>10: return True
        if calc.res<3: calc.res = calc.res*2

In [80]:
calculator = SlowCalculator()

In [81]:
calculator.calc()
calculator.res

returned
returned
returned
returned
returned
returned
returned
returned
returned
returned


30